In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, FixedLocator
from tqdm import trange
from tqdm import tqdm
from pathlib import Path
import iss_preprocess as iss
from flexiznam import PARAMETERS

In [ ]:
def plot_genes(gene_data, data_path, roi=1, plot_axis=True):
    plt.rcParams['figure.facecolor'] = 'white'
    processed_path = Path(PARAMETERS['data_root']['processed'])
    ops = iss.io.load_ops(data_path)
    roi_dims = iss.io.get_roi_dimensions(data_path)[0]
    nx = roi_dims[1]
    ny = roi_dims[2]
    dim_x = (nx*3000)+3000
    dim_y = (ny*3000)+3000
    marker = 'o'

    genes = np.sort(gene_data['Gene'].unique())
    num_genes = len(genes)

    num_cols = 3
    num_rows = math.ceil(num_genes / num_cols)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(25, 6*num_rows))

    for i, gene in tqdm(enumerate(genes), desc="Genes computed", total=num_genes):
        color = 'black'

        row_idx = i // num_cols
        col_idx = i % num_cols

        ax = axes[row_idx, col_idx]

        # Rotate the data 90 degrees counterclockwise
        rotated_x = gene_data[gene_data['Gene'] == gene]['y'].values
        rotated_y = dim_x - gene_data[gene_data['Gene'] == gene]['x'].values

        ax.plot(
            rotated_x,
            rotated_y,
            marker,
            c=color,
            markersize=0.1,
        )
        total_counts = len(rotated_x)
        ax.set_title(gene, fontsize=20)
        ax.set_xlabel(f'Total counts = {total_counts}')
        ax.set_aspect('equal', 'box')

        ax.set_xlim(0, dim_y)
        ax.set_ylim(0, dim_x)

        if plot_axis:
            ax.set_xticks(np.arange(0, dim_y, 3000)+1500)
            ax.set_yticks(np.arange(0, dim_x, 3000)+1500)
            minor_locator1 = AutoMinorLocator(2)
            minor_locator2 = FixedLocator(np.arange(0, dim_x, 3000))
            ax.xaxis.set_minor_locator(minor_locator1)
            ax.yaxis.set_minor_locator(minor_locator2)
            ax.grid(which='minor', color='lightgrey')

            ax.set_xticklabels(np.arange(0, len(ax.get_xticks())), rotation=90)
            ax.set_yticklabels(np.arange(0, len(ax.get_yticks()))[::-1])
            ax.tick_params(top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=True)

    if num_genes < num_rows * num_cols:
        for i in range(num_genes, num_rows * num_cols):
            row_idx = i // num_cols
            col_idx = i % num_cols
            fig.delaxes(axes[row_idx, col_idx])

    if not plot_axis:
        for ax in axes.flat:
            ax.axis('off')

    for ax in axes.flat:
        ax.invert_yaxis()
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.3)

    ref_tile = ops['ref_tile']
    mouse_name = (processed_path / data_path).parent.name
    chamber_name = (processed_path / data_path).name
    fig.suptitle(f'{mouse_name} {chamber_name} ROI: {roi}, ref tile: {ref_tile}', fontsize=40, y=1.01)
    save_path = processed_path / data_path / 'figures' / 'round_overviews' / 'gene_spots'
    save_path.mkdir(exist_ok=True, parents=True)
    plt.savefig(save_path / f'roi_{roi}_gene_spots.png', bbox_inches='tight')
    plt.show()


In [ ]:
data_path = 'becalia_rabies_barseq/BRAC8498.3e/chamber_09/'
roi = [1,2,3,4,5,6,7,8,9,10]

for iroi in trange(1, 11, desc="ROI", total=len(roi)):
    processed_path = iss.io.get_processed_path(data_path)
    gene_spots = pd.read_pickle(
        processed_path / f'genes_round_spots_{iroi}.pkl')
    gene_spots.rename(columns={'gene':'Gene'}, inplace=True)
    gene_spots = gene_spots[gene_spots['spot_score']>0.2]
    plot_genes(gene_spots, data_path, roi=iroi, plot_axis=True)